In [5]:
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
import pandas_datareader.data as web
from scipy.optimize import minimize
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [6]:
ticker_list = ['SOL-USD','ETH-USD','BTC-USD']

In [7]:
start_date = dt.datetime(2011, 1, 1)
end_date = dt.datetime(2024, 12, 5)
returns = (yf.download(ticker_list,start_date-pd.offsets.BDay(1),end_date+pd.offsets.BDay(1))['Adj Close']
               .resample('W')
               .last()
               .pct_change().dropna())
returns.index=returns.index.tz_localize(None)
returns


*********************100%***********************]  3 of 3 completed

Ticker,BTC-USD,ETH-USD,SOL-USD
Date,,,
2020-04-19,0.031320,0.127046,-0.312222
2020-04-26,0.068217,0.086461,0.047016
2020-05-03,0.158545,0.069004,0.069282
2020-05-10,-0.015851,-0.105880,-0.209704
2020-05-17,0.104416,0.098405,0.141434
...,...,...,...
2024-11-10,0.170685,0.299177,0.295675
2024-11-17,0.116456,-0.036245,0.128015
2024-11-24,0.090911,0.093638,0.064586


In [13]:
from statsmodels.tsa.arima.model import ARIMA
from datetime import datetime

In [14]:
def forecast_timeseries(data, forecast_steps=30):
    """
    Forecast Bitcoin prices using an ARIMA model.
    :param data: DataFrame containing historical prices with a 'Date' and 'Adj Close' column.
    :param forecast_steps: Number of days to forecast.
    :return: Forecasted values and the model summary.
    """
    # Convert 'Date' to datetime and set as index
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)

    # Ensure the data is sorted by date
    data = data.sort_index()

    # Extract the 'Adj Close' column for forecasting
    adj_close = data['Adj Close']

    # Fit the ARIMA model (p, d, q) parameters can be tuned
    model = ARIMA(adj_close, order=(5, 1, 0))  # Example: ARIMA(5, 1, 0)
    model_fit = model.fit()

    # Forecast future values
    forecast = model_fit.forecast(steps=forecast_steps)

    # Create a DataFrame for the forecast
    forecast_dates = pd.date_range(start=adj_close.index[-1] + pd.Timedelta(days=1), periods=forecast_steps)
    forecast_df = pd.DataFrame({'Date': forecast_dates, 'Forecasted Price': forecast})
    
    return forecast_df, model_fit

In [12]:
forecast_steps = 30  # Forecast for the next 30 days
forecast_df, model_fit = forecast_timeseries(data, forecast_steps)

forecast_df.to_csv('bitcoin_forecast.csv', index=False)
print(forecast_df)

plt.figure(figsize=(10, 6))
plt.plot(data['Date'], data['Adj Close'], label='Historical Prices')
plt.plot(forecast_df['Date'], forecast_df['Forecasted Price'], label='Forecasted Prices', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Bitcoin Price Forecast')
plt.legend()
plt.grid()
plt.show()


NameError: name 'data' is not defined